In [48]:
!pip install llama-index openai
!pip install -q --progress-bar off llama-index-readers-docling \
llama-index-node-parser-docling llama-index-embeddings-huggingface \
llama-index-llms-huggingface-api llama-index-vector-stores-milvus \
llama-index-readers-file
!pip install  --upgrade openai
1+1


I0000 00:00:1751518161.267915 3806843 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [34]:
!pip install  llama_index.core.chat_engine 


I0000 00:00:1751622002.719100 6369203 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


ERROR: Could not find a version that satisfies the requirement llama_index.core.chat_engine (from versions: none)
ERROR: No matching distribution found for llama_index.core.chat_engine


In [36]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from dotenv import load_dotenv
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.memory import ChatMemoryBuffer


load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:

def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


GEN_MODEL = HuggingFaceInferenceAPI(
    token=os.getenv("HF_TOKEN"),
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

In [33]:
#""
'''BAAI/bge-small-en-v1.5 - Beijing Academy of Artificial Intelligence (BAAI)
Sentence Embedding Model / Text Embedding Model
Specifically designed for English text.
Based on E5 architecture, which itself is a modification of the MiniLM 
(or similar lightweight Transformer) architecture optimized for embedding tasks.
~60 million parameters'''

EMBED_MODEL = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")
embed_dim = len(EMBED_MODEL.get_text_embedding("Burger"))#
print(embed_dim)

384


In [8]:
SOURCE = 'Internal.pdf'
#SOURCE = "https://arxiv.org/pdf/2408.09869"

In [9]:
reader = DoclingReader()
node_parser = MarkdownNodeParser()
MILVUS_URI = str(Path(mkdtemp())/ 'docling_ahtsham.db')
vector_store = MilvusVectorStore(uri=MILVUS_URI,dim=embed_dim,overwrite=True)
index = VectorStoreIndex.from_documents(
    documents=reader.load_data(SOURCE),
    transformations=[node_parser],
    storage_context=StorageContext.from_defaults(vector_store=vector_store),
    embed_model=EMBED_MODEL,
)


/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-07-04 10:32:15,535 [DEBUG][_create_connection]: Created new connection using: 16eb007264024252a77f03cd7460e4b7 (async_milvus_client.py:599)


In [21]:
models = ['gpt-3.5-turbo','text-davinci-003']

llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=512
)



In [28]:


#QUERY =  'How do you ensure software compliance (licensing) in an organization '
QUERY = "What tools and metrics would you use to monitor server hardware performance?"
result = index.as_query_engine(llm=llm).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])

Q: What tools and metrics would you use to monitor server hardware performance?
A: Use Nagios, Zabbix, Prometheus, or cloud tools like AWS CloudWatch or Azure Monitor to monitor server hardware performance. Metrics to monitor include CPU usage, RAM usage, disk I/O (throughput/latency), disk space, network throughput, and temperatures. SMART data can be used to monitor disk health indicators such as reallocated sectors and error rates. SNMP can be enabled on devices to collect performance data, and logs like syslog or Event Viewer can be centralized to catch hardware errors like fan failures or power issues. Configure dashboards for real-time viewing and set alerts for threshold breaches, such as CPU exceeding 90% or temperature exceeding 75°C. Capacity planning involves analyzing trends to anticipate upgrades, like disk space nearing capacity.

Sources:


[("## Hardware Troubleshooting\n\n- Question: How do you diagnose a failing hard disk in a server? 1.\n- Answer: Disk failures often cause slowdowns or errors. Troubleshooting steps:\n- SMART Data: Use smartctl or a RAID controller utility to check SMART status (reallocated sectors, pending errors). 2.\n- Noise Check: Listen for unusual sounds (clicking, beeping). 3.\n- Log Review: Check system logs (Windows Event Viewer or /var/log/ ) for disk I/O errors. 4.\n- Disk Check: Run file system checks (CHKDSK on Windows, fsck on Linux) to find bad sectors. 5.\n- Backup Immediately: Copy critical data to backup storage at first sign of failure. 6.\n\n- Replace Disk: If   errors   persist,   replace the drive and rebuild/restore from backups or RAID parity. 7.\n- Question: Describe the differences between RAID 0, 1, 5, and 10 in terms of redundancy and performance. 8.\n\nAnswer: RAID levels offer different trade-offs:\n\n- RAID 0 (Striping): No redundancy. Data is striped across disks for hig

In [41]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

custom_prompt = PromptTemplate(
    "You are an expert IT assistant. Use the context below to answer the user's question. "
    "If the answer in the context is written in bullet points or numbered lists, preserve that format in your answer.\n\n"
    "Context:\n{context_str}\n\n"
    "Question: {query_str}\n"
    "Answer:"
)

query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(),
    text_qa_template=custom_prompt,
    llm=llm,
)

In [48]:
response = query_engine.query(" Provide a JSON-formatted example of a network device configuration for documentation")
print(response)

{
  "device_name": "BranchRouter",
  "device_type": "Router",
  "interfaces": [
    {
      "name": "GigabitEthernet0/0",
      "ip_address": "192.168.1.1",
      "subnet_mask": "255.255.255.0",
      "description": "LAN Interface"
    },
    {
      "name": "GigabitEthernet0/1",
      "ip_address": "10.10.20.1",
      "subnet_mask": "255.255.255.0",
      "description": "Connection to Remote Network"
    }
  ],
  "routing": [
    {
      "destination_network": "192.168.50.0/24",
      "next_hop": "192.168.1.254",
      "interface": "GigabitEthernet0/0"
    },
    {
      "destination_network": "10.10.20.0/24",
      "next_hop": "10.10.20.254",
      "interface": "GigabitEthernet0/1"
    }
  ],
  "firewall_rules": [
    {
      "source": "192.168.1.0/24",
      "destination": "10.10.20.0/24",
      "allow": true,
      "description": "Allow traffic from LAN to Remote Network"
    },
    {
      "source": "any",
      "destination": "any",
      "allow": false,
      "description": "Den

In [40]:
response = chat_engine.chat([
    {"role": "system", "content": "You are an expert DevOps assistant."},
    {"role": "user", "content": f"{QUERY}"}
])

print(response.response)

Querying with: [{'role': 'system', 'content': 'You are an expert DevOps assistant.'}, {'role': 'user', 'content': 'What tools and metrics would you use to monitor server hardware performance?'}]


ValidationError: 2 validation errors for QueryStartEvent
query.str
  Input should be a valid string [type=string_type, input_value=[{'role': 'system', 'cont...hardware performance?'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
query.QueryBundle
  Input should be a dictionary or an instance of QueryBundle [type=dataclass_type, input_value=[{'role': 'system', 'cont...hardware performance?'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/dataclass_type

In [29]:
result.source_nodes

[NodeWithScore(node=TextNode(id_='7ebcb2d7-ea89-4a41-baac-60b58b1ee34e', embedding=None, metadata={'header_path': '/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7160dd5f-bb88-47b9-baaf-56d7d65fea07', node_type='4', metadata={}, hash='9f213321a00b85bf4435f6adc3dabeae885b47083972dd8ad7c7500b1e3c511a'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5028152b-3b5d-44a4-99ce-8f5b51074a23', node_type='1', metadata={'header_path': '/'}, hash='257a0897aa55c114fee09468378d025c6963b9fe88f25e6c77f82833dec64e67'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='bda52f14-5a91-4dbc-9c44-fd0fb60b4ab1', node_type='1', metadata={'header_path': '/'}, hash='f08576f9e50072819042462c9305fdd2e1f540142bc22055d4f64b7cdc7978b3')}, metadata_template='{key}: {value}', metadata_separator='\n', text="## Hardware Troubleshooting\n\n- Question: How do you diagnose a failing hard disk in a server? 1.\n- A

llama_index.core.indices.vector_store.base.VectorStoreIndex

In [ ]:
1+1

In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(token=os.getenv("HF_TOKEN"))
response = client.text_generation(
    prompt="How old was albert einstein when he died?",
    model="google/flan-t5-base",
    max_new_tokens=100
)
print(response)

In [14]:

GEN_MODEL = HuggingFaceInferenceAPI(
    token=os.getenv("HF_TOKEN"),
    model_name="HuggingFaceH4/zephyr-7b-beta",)

QUERY =  'How old was albert einstein when he died?'
result = index.as_query_engine(llm=GEN_MODEL).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])

HfHubHTTPError: 404 Client Error: Not Found for url: https://router.huggingface.co/hf-inference/models/HuggingFaceH4/zephyr-7b-beta/v1/chat/completions (Request ID: Root=1-6867631b-73cd515207d00dd6605e66ac;ee8bba0d-649b-463a-8593-7644c55ef6fe)

In [15]:
import os
from huggingface_hub import InferenceClient
import requests

# Solution 1: Use InferenceClient (Recommended)
def solution_1_inference_client():
    """Using the official InferenceClient"""
    try:
        client = InferenceClient(token=os.getenv("HF_TOKEN"))
        
        # Try different model names that are known to work
        models_to_try = [
            "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "mistralai/Mistral-7B-Instruct-v0.1",
            "microsoft/DialoGPT-medium",
            "google/flan-t5-base",
            "HuggingFaceH4/zephyr-7b-beta"
        ]
        
        QUERY = 'How old was albert einstein when he died?'
        
        for model_name in models_to_try:
            try:
                print(f"Trying model: {model_name}")
                
                # For chat models
                if "instruct" in model_name.lower() or "chat" in model_name.lower():
                    messages = [{"role": "user", "content": QUERY}]
                    response = client.chat_completion(
                        messages=messages,
                        model=model_name,
                        max_tokens=500
                    )
                    result = response.choices[0].message.content
                else:
                    # For text generation models
                    response = client.text_generation(
                        prompt=QUERY,
                        model=model_name,
                        max_new_tokens=500
                    )
                    result = response
                
                print(f"Q: {QUERY}")
                print(f"A: {result}")
                return result
                
            except Exception as e:
                print(f"Error with {model_name}: {e}")
                continue
        
        print("All models failed to respond")
        return None
        
    except Exception as e:
        print(f"Error setting up InferenceClient: {e}")
        return None

# Solution 2: Direct API call with requests
def solution_2_direct_api():
    """Direct API call to HuggingFace"""
    headers = {
        "Authorization": f"Bearer {os.getenv('HF_TOKEN')}",
        "Content-Type": "application/json"
    }
    
    # Models that are more likely to work with the inference API
    models_to_try = [
        "mistralai/Mistral-7B-Instruct-v0.1",
        "microsoft/DialoGPT-medium",
        "google/flan-t5-base",
        "HuggingFaceH4/zephyr-7b-beta"
    ]
    
    QUERY = 'How old was albert einstein when he died?'
    
    for model_name in models_to_try:
        try:
            print(f"Trying model: {model_name}")
            
            url = f"https://api-inference.huggingface.co/models/{model_name}"
            
            payload = {
                "inputs": QUERY,
                "parameters": {
                    "max_new_tokens": 500,
                    "temperature": 0.7
                }
            }
            
            response = requests.post(url, headers=headers, json=payload)
            
            if response.status_code == 200:
                result = response.json()
                if isinstance(result, list) and len(result) > 0:
                    answer = result[0].get('generated_text', '')
                    print(f"Q: {QUERY}")
                    print(f"A: {answer}")
                    return answer
                else:
                    print(f"Unexpected response format: {result}")
            else:
                print(f"Error {response.status_code}: {response.text}")
                
        except Exception as e:
            print(f"Error with {model_name}: {e}")
            continue
    
    print("All models failed to respond")
    return None

# Solution 3: Using a working model for your LlamaIndex setup
def solution_3_llamaindex_compatible():
    """Code that should work with your LlamaIndex setup"""
    from llama_index.llms.huggingface import HuggingFaceLLM
    
    # Alternative approach using HuggingFaceLLM instead of HuggingFaceInferenceAPI
    try:
        # Use a model that's known to work
        llm = HuggingFaceLLM(
            model_name="microsoft/DialoGPT-medium",
            tokenizer_name="microsoft/DialoGPT-medium",
            query_wrapper_prompt="<|User|>: {query_str}\n<|Assistant|>: ",
            context_window=2048,
            max_new_tokens=512,
            model_kwargs={"torch_dtype": "auto"},
            tokenizer_kwargs={},
            device_map="auto",
        )
        
        QUERY = 'How old was albert einstein when he died?'
        
        # If you have an index setup
        # result = index.as_query_engine(llm=llm).query(QUERY)
        # print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
        # display([(n.text, n.metadata) for n in result.source_nodes])
        
        # For direct testing without index
        response = llm.complete(QUERY)
        print(f"Q: {QUERY}")
        print(f"A: {response.text}")
        
        return response.text
        
    except Exception as e:
        print(f"Error with HuggingFaceLLM: {e}")
        return None

# Solution 4: Simple working example with a free model
def solution_4_simple_working():
    """Simple working example with a free model"""
    from huggingface_hub import InferenceClient
    
    try:
        client = InferenceClient(token=os.getenv("HF_TOKEN"))
        
        # Use a model that's definitely available
        model_name = "google/flan-t5-base"
        QUERY = 'How old was albert einstein when he died?'
        
        response = client.text_generation(
            prompt=f"Question: {QUERY}\nAnswer:",
            model=model_name,
            max_new_tokens=100
        )
        
        print(f"Q: {QUERY}")
        print(f"A: {response}")
        return response
        
    except Exception as e:
        print(f"Error: {e}")
        return None

# Main execution
if __name__ == "__main__":
    print("=== Hugging Face Inference API Fix ===\n")
    
    # Check if HF_TOKEN is set
    if not os.getenv("HF_TOKEN"):
        print("❌ HF_TOKEN environment variable not set!")
        print("Please set it with: export HF_TOKEN='your_token_here'")
        exit(1)
    
    print("🔧 Trying different solutions...\n")
    
    # Try solutions in order
    solutions = [
        ("Solution 1: InferenceClient", solution_1_inference_client),
        ("Solution 2: Direct API", solution_2_direct_api),
        ("Solution 3: LlamaIndex Compatible", solution_3_llamaindex_compatible),
        ("Solution 4: Simple Working", solution_4_simple_working)
    ]
    
    for name, func in solutions:
        print(f"\n{'='*50}")
        print(f"🧪 {name}")
        print('='*50)
        
        result = func()
        if result:
            print(f"✅ {name} worked!")
            break
        else:
            print(f"❌ {name} failed")
    
    print("\n" + "="*50)
    print("💡 Tips:")
    print("1. Some models require paid HF Pro subscription")
    print("2. Try smaller models like 'google/flan-t5-base'")
    print("3. Check model availability at https://huggingface.co/models")
    print("4. Consider using OpenAI API as alternative")

=== Hugging Face Inference API Fix ===

🔧 Trying different solutions...


🧪 Solution 1: InferenceClient
Trying model: mistralai/Mixtral-8x7B-Instruct-v0.1
Q: How old was albert einstein when he died?
A:  Albert Einstein died at the age of 76. He lived a long and impactful life, contributing significantly to the field of theoretical physics and changing the way we understand time, space, energy, and matter. Einstein is best known for his theory of relativity, which includes the famous equation E=mc², demonstrating the equivalence of energy (E) and mass (m) when coupled with the speed of light (c). Despite his many accomplishments and accolades, Einstein remained humble and continued to be curious and passionate about the mysteries of the universe until the end of his life.
✅ Solution 1: InferenceClient worked!

💡 Tips:
1. Some models require paid HF Pro subscription
2. Try smaller models like 'google/flan-t5-base'
3. Check model availability at https://huggingface.co/models
4. Consider 